In [100]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
!pip install tabulate

In [47]:
evc_df = pd.read_csv('../data/processed/evc_df.csv')
vfacts_df = pd.read_csv('../data/processed/vfacts_df.csv')
nsw_df = pd.read_csv('../data/processed/nsw_rego_data.csv')
abs_df = pd.read_csv('../data/processed/abs_df.csv')
bitre_df = pd.read_csv('../data/processed/bitre_df.csv')

In [109]:
PC_passenger_ev_sales_pc['2023-01-31'].values

array([24.20656267,  0.        ,  0.        , ...,  0.        ,
        0.        ,  0.        ])

In [118]:
display(S_ev_pc.loc['Passenger',3221])
display(S_all_pc.loc['Passenger',3221])

2021-01-31     3.0
2022-01-31     4.0
2023-01-31    10.0
Name: (Passenger, 3221), dtype: float64

2021-01-31    3131.0
2022-01-31    2892.0
2023-01-31    2730.0
Name: (Passenger, 3221), dtype: float64

In [131]:
(S_all_pc.diff(axis=1) / (S_ev_pc*0+1))>0

2021-01-31  2022-01-31  2023-01-31
Vehicle Type     Postcode                                    
Light Commercial 800            False       False       False
                 801            False       False       False
                 803            False       False       False
                 810            False       False       False
                 811            False       False       False
...                               ...         ...         ...
Passenger        8011           False       False       False
                 8012           False       False       False
                 8051           False       False       False
                 9382           False       False       False
                 9999           False       False       False

[5627 rows x 3 columns]

In [135]:
bitre_df['Vehicle Type'].unique()
bitre_df['Fuel Type'].unique()
S_all_pc = bitre_df.groupby(['Vehicle Type','Postcode']).sum().drop(columns=['State','Fuel Type'])
S_all_nat = S_all_pc.groupby(['Vehicle Type']).sum()
S_ev_pc = bitre_df[bitre_df['Fuel Type']=='Electric'].groupby(['Vehicle Type','Postcode']).sum().drop(columns=['State','Fuel Type'])
S_ev_nat = S_ev_pc.groupby(['Vehicle Type']).sum()
scrapping_rate = [0.06,0.06,0.06]
PC_ev_sales_nat = S_ev_nat.diff(axis=1) / (S_all_nat.diff(axis=1) + scrapping_rate * (S_all_nat - S_ev_nat)) * 100
PC_passenger_ev_sales_nat = PC_ev_sales_nat.loc['Passenger']
PC_ev_sales_pc = S_ev_pc.diff(axis=1) / (S_all_pc.diff(axis=1) + scrapping_rate * (S_all_pc - S_ev_pc)) * 100
PC_ev_sales_pc[(S_all_pc.diff(axis=1) / (S_ev_pc*0+1))<0] = float('NaN') # ignore when the stock is going down
PC_passenger_ev_sales_pc = PC_ev_sales_pc.loc['Passenger']
PC_passenger_ev_sales_pc = PC_passenger_ev_sales_pc.fillna(0) # nans are caused by not reporting postcodes where ev stock is zero
PC_passenger_ev_sales_pc[PC_passenger_ev_sales_pc < 0] = 0
PC_passenger_ev_sales_pc[PC_passenger_ev_sales_pc > 100] = float('NaN') # ignore when value is unreasonable high
print(PC_passenger_ev_sales_pc.head(1000).to_markdown())
# plot histogram of PC_passenger_ev_sales_pc
fig = go.Figure(data=[go.Histogram(x=PC_passenger_ev_sales_pc['2023-01-31'].values,xbins=dict(start=0.0, end=20, size=0.5))])
fig.show()
# perhaps set the uncertainty in any fit to these percentages as a function of the stock in that postcode?

|   Postcode |   2021-01-31 |   2022-01-31 |   2023-01-31 |
|-----------:|-------------:|-------------:|-------------:|
|        800 |            0 |    0.263699  |     0        |
|        801 |            0 |    0         |     0        |
|        803 |            0 |    0         |     0        |
|        804 |            0 |    0         |     0        |
|        810 |            0 |    0.713643  |     0        |
|        811 |            0 |    0         |     0        |
|        812 |            0 |    0         |     0        |
|        813 |            0 |    0         |     0        |
|        814 |            0 |    0         |     0        |
|        815 |            0 |    0         |     0        |
|        819 |            0 |    0         |     0        |
|        820 |            0 |    0         |     1.53766  |
|        821 |            0 |    0         |     0        |
|        822 |            0 |    0         |     1.28337  |
|        828 |            0 |    2.05761

In [84]:
def find_first_date_idx(df):
    dt = pd.to_datetime(df.columns, errors='coerce', format='%Y-%m-%d')
    is_datetime = pd.Series(dt).notnull()
    # find index of first true in is_datetime
    idx = is_datetime[is_datetime].index[0]
    return idx

def rolling_sum(values, period):
    rolling = []
    for i in range(0, len(values)):
        num_sum = period + min(i-period+1,0)
        rolling.append(values[i-(num_sum-1):i+1].sum()/num_sum)
    return rolling

def get_yearly_from_quarterly(df):
    
        fc = find_first_date_idx(df)
    
        yearly_df = df.copy()
    
        for row in range(0, len(df)):
            # get values from row
            values = df.iloc[row,fc:].values
            # quarterly to yearly
            yearly = rolling_sum(values, 4)
            yearly_df.iloc[row,fc:] = yearly
    
        visualize = False 
        if visualize:
            # plotly line plot of vte with x-axis as datetime and y-axis as vte
            row = 1
            q_values = df.iloc[row,fc:].values
            y_values = yearly_df.iloc[row,fc:].values
    
            index = df.iloc[row,fc:].index
    
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=index, y=q_values, mode='lines+markers'))
            fig.add_trace(go.Scatter(x=index, y=y_values, mode='lines+markers'))
            fig.show()
        
        return yearly_df

def get_quarterly_and_yearly_from_monthly(df):

    fc = find_first_date_idx(df)

    quarterly_df = df.copy()
    yearly_df = df.copy()

    for row in range(0, len(df)):
        # get values from row
        values = df.iloc[row,fc:].values
        # monthly to quarterly
        quarterly = rolling_sum(values, 3)
        quarterly_df.iloc[row,fc:] = quarterly

        # quarterly to yearly
        yearly = rolling_sum(values, 12)
        yearly_df.iloc[row,fc:] = yearly

    visualize = False
    if visualize:
        # plotly line plot of vte with x-axis as datetime and y-axis as vte
        row = 1
        m_values = df.iloc[row,fc:].values
        q_values = quarterly_df.iloc[row,fc:].values
        y_values = yearly_df.iloc[row,fc:].values

        index = df.iloc[row,fc:].index

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=index, y=m_values, mode='lines+markers'))
        fig.add_trace(go.Scatter(x=index, y=q_values, mode='lines+markers'))
        fig.add_trace(go.Scatter(x=index, y=y_values, mode='lines+markers'))
        fig.show()
    
    return quarterly_df, yearly_df

nsw_quarterly_df, nsw_yearly_df = get_quarterly_and_yearly_from_monthly(nsw_df)
vfacts_yearly_df = get_yearly_from_quarterly(vfacts_df)

In [85]:
def vf_elec_percent_nsw(vdf):
    vt = vdf.loc[vdf['State']=='NSW'].groupby(['Fuel Type']).sum().drop(columns=['State','Vehicle Type'])
    vt = vt/vt.sum()*100
    vte = vt.loc['Electric']
    return vte

def vf_elec_percent_national(vdf):
    vt = vdf.groupby(['Fuel Type']).sum().drop(columns=['State','Vehicle Type'])
    vt = vt/vt.sum()*100
    vte = vt.loc['Electric']
    return vte

q_vte_nsw = vf_elec_percent_nsw(vfacts_df)
y_vte_nsw = vf_elec_percent_nsw(vfacts_yearly_df)

q_vte_nat = vf_elec_percent_national(vfacts_df)
y_vte_nat = vf_elec_percent_national(vfacts_yearly_df)

# plotly line plot of vte with x-axis as datetime and y-axis as vte
fig = go.Figure()
fig.add_trace(go.Scatter(x=q_vte_nsw.index, y=q_vte_nsw.values, mode='lines+markers',name='NSW VFACTS Sales (quarterly)'))
fig.add_trace(go.Scatter(x=y_vte_nsw.index, y=y_vte_nsw.values, mode='lines+markers',name='NSW VFACTS Sales (yearly)'))
fig.add_trace(go.Scatter(x=q_vte_nat.index, y=q_vte_nat.values, mode='lines+markers',name='Nat VFACTS Sales (quarterly)'))
fig.add_trace(go.Scatter(x=y_vte_nat.index, y=y_vte_nat.values, mode='lines+markers',name='Nat VFACTS Sales (yearly)'))
fig.add_trace(go.Scatter(x=PC_passenger_ev_sales_nat.index, y=PC_passenger_ev_sales_nat.values, mode='lines+markers',name='BITRE Sales (yearly)'))
fig.update_layout(title='Electric Vehicle Sales as a Percentage of Total Vehicle Sales',
                  xaxis_title='Date',
                  yaxis_title='Percentage of Total Vehicle Sales')
fig.show()

In [86]:
et = evc_df.groupby(['Fuel Type']).sum()
ete = et.loc['Electric'] / et.loc['All'] * 100

# overlay ete on fig plot
fig.add_trace(go.Scatter(x=ete.index, y=ete.values, mode='lines+markers',name='National EVC (yearly)'))
fig.update_layout(title='Electric Vehicle Sales as a Percentage of Total Vehicle Sales',
                  xaxis_title='Date',
                  yaxis_title='Percentage of Total Vehicle Sales')
fig.show()

In [52]:
def nsw_elec_percent(ndf):
    nt = ndf.groupby(['Fuel Type']).sum().drop(columns=['State','Vehicle Type'])
    nte = nt.loc['Electric']/nt.loc['All']*100
    return nte

m_nte = nsw_elec_percent(nsw_df)
q_nte = nsw_elec_percent(nsw_quarterly_df)
y_nte = nsw_elec_percent(nsw_yearly_df)

# overlay ete on fig plot
fig.add_trace(go.Scatter(x=m_nte.index, y=m_nte.values, mode='lines+markers',name = 'NSW New Regos (monthly)'))
fig.add_trace(go.Scatter(x=q_nte.index, y=q_nte.values, mode='lines+markers',name = 'NSW New Regos (quarterly)'))
fig.add_trace(go.Scatter(x=y_nte.index, y=y_nte.values, mode='lines+markers',name = 'NSW New Regos (yearly)'))
fig.update_layout(title='Electric Vehicle Sales as a Percentage of Total Vehicle Sales',
                  xaxis_title='Date',
                  yaxis_title='Percentage of Total Vehicle Sales')
fig.show()
